# Scripts tests

Just a test of the modules written to schedule the collection of data from Twitch, storing it on disk, and/or sending it to Mongo.

## Imports

In [1]:
from ast import literal_eval
from bson.son import SON
import json
import os
import sys
from pymongo import MongoClient
from bson.objectid import ObjectId

Also import the modules with the functions for collection (the names are provisional)

List of python modules needed:
1. apscheduler
2. ast
3. datetime
4. json
5. os
6. pymongo
7. requests

Add folder with Python scripts to path:

In [2]:
sys.path.insert(0, '../scripts')

# Twitch Script Test

A Twitch Client ID is needed to run the scrpit.

In [3]:
import twitch_collect_schedule

In [4]:
filename = "../keys.json"
with open(filename) as file:
    keys = json.loads(file.read())
    twitch_client_ID = keys['Twitch']['Client-ID']

header_v5 = {
    'Accept': 'application/vnd.twitchtv.v5+json',
    'Client-ID': twitch_client_ID,
}

The Twitch script, named twitch_collect_schedule.py, collects data from Twitch about top games. The data is collected every 30 seconds by default, but the interval time can be specified.

The collected data can be saved on disk as a json file, sent to a MongoDB collection, or both. Here we will just send it to a local test MongoDB.

Loading Mongo's client

In [5]:
client = MongoClient('localhost', 27017)

Using the db 'twitchtest'.

In [6]:
db = client.twitchtest

And the collection 'games'

In [7]:
games = db.games

We now start the script to collect data from twitch and sending it to the mongo collection. Once started, the script needs to be interrupted manually (which takes some time on jupyter).

For testing purposes, in order to collect data quicker, we will use an interval of 20 seconds.

In [8]:
twitch_collect_schedule.twitch_collector_scheduler(header_v5, games, trigger = 'interval',  seconds=20,
                                                   print_progress=True, save_local=False)

Script started at: 2019-07-06 10:45:26.587840

Press Ctrl+Break to exit
Job started at 2019-07-06 10:45:26.637711

Games processed: 199
Games processed: 299
Games processed: 399
Games processed: 499
Games processed: 599
Games processed: 699
Games processed: 799
Games processed: 899
Games processed: 999
Games processed: 1099
Games processed: 1199
Games processed: 1297
Games processed: 1396
Games processed: 1496
Games processed: 1593
Games processed: 1614
Done collecting!
Sent to Mongo: 5d207bd1f0bd061fec6ca6b4
Job completed at 2019-07-06 10:45:38.020711



If print_progress = True, the script also prints the ids of the data sent. E.g., in our case, '5d207bd1f0bd061fec6ca6b4'.

After stopping the script, we can look for it as follows.

In [9]:
test = list(games.find({'_id' : ObjectId('5d207bd1f0bd061fec6ca6b4')}))

In [11]:
test[0]['data'][0]

{'channels': 1723,
 'game__id': 21779,
 'game_box_large': 'https://static-cdn.jtvnw.net/ttv-boxart/League%20of%20Legends-272x380.jpg',
 'game_giantbomb_id': 24024,
 'game_logo_large': 'https://static-cdn.jtvnw.net/ttv-logoart/League%20of%20Legends-240x144.jpg',
 'game_name': 'League of Legends',
 'game_norm_name': 'league of legends',
 'game_popularity': 110697,
 'viewers': 114859}

# Twitter Script Test